In [1]:
import numpy as np
import pandas as pd
import os

# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
print("UYARI ENGELLEYİCİ")

UYARI ENGELLEYİCİ


In [2]:
filepath = '/kaggle/input/commonlit-evaluate-student-summaries'

prompts_test = pd.read_csv(os.path.join(filepath, 'prompts_test.csv'))
prompts_train = pd.read_csv(os.path.join(filepath, 'prompts_train.csv'))
sub = pd.read_csv(os.path.join(filepath, 'sample_submission.csv'))
summaries_test = pd.read_csv(os.path.join(filepath, 'summaries_test.csv'))
summaries_train = pd.read_csv(os.path.join(filepath, 'summaries_train.csv'))
print("DOSYA YOL KODLARI")

DOSYA YOL KODLARI


In [3]:
import nltk
# NLTK'den tokenizer'ı yükle
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Tokenizasyon işlemi
summaries_train['tokenized_text'] = summaries_train['text'].apply(lambda x: word_tokenize(x))
prompts_test['prompt_text'] = prompts_test['prompt_text'].apply(lambda x: word_tokenize(x))
summaries_test['text'] = summaries_test['text'].apply(lambda x: word_tokenize(x))

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [4]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re

# NLTK stopwords verilerini indirin
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Temizleme işlemlerini uygulama fonksiyonu
def preprocess_text(tokenized_text):
    # Her bir kelimeyi temizle
    cleaned_words = []
    for word in tokenized_text:
        # Küçük harfe dönüştürme
        word = word.lower()
        # Noktalama işaretlerini boşlukla değiştirme
        word = re.sub(r'[^\w\s]', ' ', word)
        # Gereksiz boşlukları kaldırma
        word = re.sub(r'\s+', ' ', word)
        # Stopwords'leri kaldırma
        if word not in stop_words:
            cleaned_words.append(word)
    return ' '.join(cleaned_words)

# Tokenleştirilmiş metinleri temizleme
summaries_train['cleaned_text'] = summaries_train['tokenized_text'].apply(preprocess_text)
prompts_test['prompt_text'] = prompts_test['prompt_text'].apply(preprocess_text)
summaries_test['text'] = summaries_test['text'].apply(preprocess_text)

# Noktalama işaretlerini boşlukla değiştirme
summaries_train['cleaned_text'] = summaries_train['cleaned_text'].str.replace('[^\w\s]', '')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('[^\w\s]', '')
summaries_test['text'] = summaries_test['text'].str.replace('[^\w\s]', '')

# ŞİMDİ BOŞLUKLARI KALDIRRRR HAYDİİİİİİİİİİİ
summaries_train['cleaned_text'] = summaries_train['cleaned_text'].str.replace('\s+', '')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('\s+', '')
summaries_test['text'] = summaries_test['text'].str.replace('\s+', '')

#Bişey deniyorum
summaries_train['cleaned_text'] = summaries_train['cleaned_text'].str.replace('  ', ' ')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('  ', ' ')
summaries_test['text'] = summaries_test['text'].str.replace('  ', ' ')

summaries_train['cleaned_text'] = summaries_train['cleaned_text'].str.replace('   ', ' ')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('   ', ' ')
summaries_test['text'] = summaries_test['text'].str.replace('   ', ' ')

summaries_train['cleaned_text'] = summaries_train['cleaned_text'].str.replace('    ', ' ')
prompts_test['prompt_text'] = prompts_test['prompt_text'].str.replace('    ', ' ')
summaries_test['text'] = summaries_test['text'].str.replace('    ', ' ')

# Temizlenmiş verileri gösterme
print(summaries_train['cleaned_text'])

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
0       third wave experimentto see people reacted new...
1       would rub soda make smell go away wouldnt bad ...
2       egypt  many occupations social classes involve...
3       highest class pharaohs people gods then 2nd hi...
4       third wave developed rapidly students genuinly...
                              ...                        
7160    used sorts chemical concoctions make meat seem...
7161    lowest classes slaves farmers slaves people ta...
7162    sorta made people start working structour they...
7163    ideal tragety three elements make ideal  start...
7164    meat would smell sour would  rub soda take awa...
Name: cleaned_text, Length: 7165, dtype: object


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF vektörleştirmeyi yapılandırın
tfidf_vectorizer = TfidfVectorizer(max_features=5000) # Örnek olarak, en yaygın 1000 terimi kullanacağız
tfidf_vectorizer_test = TfidfVectorizer(max_features=5000)

# TF-IDF matrisini oluşturun
tfidf_matrix = tfidf_vectorizer.fit_transform(summaries_train['cleaned_text'])
tfidf_matrix_test = tfidf_vectorizer_test.fit_transform(summaries_test['text'])

# TF-IDF matrisini bir DataFrame olarak saklayabilirsiniz (isteğe bağlı)
tf_idf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tf_idf_test = pd.DataFrame(tfidf_matrix_test.toarray(), columns=tfidf_vectorizer_test.get_feature_names_out())

# TF-IDF matrisini inceleyin
print(tf_idf_test)

    example      text
0  0.707107  0.707107
1  0.707107  0.707107
2  0.707107  0.707107
3  0.707107  0.707107


In [6]:
# Terimlerin sıklıklarını hesaplayın ve sıralayın
term_frequencies = tf_idf.sum().sort_values(ascending=False)

# En yaygın terimleri gösterin (örneğin, ilk 10 terimi alalım)
top_terms = term_frequencies.head(5)

# Sonuçları gösterin
print(top_terms)

would      436.117878
meat       407.681365
tragedy    277.687854
good       218.219429
spoiled    210.656793
dtype: float64


In [7]:
tf_idf.head(1)

,10,100,11,12,13,14,15,16,17,18,...,yearly,years,yes,yet,you,youd,young,youre,zero,ziggurat
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
tf_idf.shape

(7165, 5000)

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# X ve Y'yi ayırın
X = tf_idf
y = summaries_train[['content', 'wording']]

# Veriyi eğitim ve test kümelerine ayırın
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lineer regresyon modelini oluşturun
model = LinearRegression()

# Modeli eğitin
model.fit(X_train, y_train)

# Modelin test verileri üzerinde performansını değerlendirin
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R-squared:", r2)

def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

def compt_score(content_true, content_pred, wording_true, wording_pred):
    content_score = mean_squared_error(content_true, content_pred)**(1/2)
    wording_score = mean_squared_error(wording_true, wording_pred)**(1/2)
    
    return (content_score + wording_score)/2

# Örnek tahminler
random_preds = np.random.rand(len(y_test), 2)  # 2 sütunlu tahminler

# MCRMSE hesaplama fonksiyonunu kullanarak sonuçları alın
mcrmse_result = compute_mcrmse((random_preds, y_test.values))

# MCRMSE sonuçlarını yazdırın
print("Content RMSE:", mcrmse_result["content_rmse"])
print("Wording RMSE:", mcrmse_result["wording_rmse"])
print("MCRMSE:", mcrmse_result["mcrmse"])

print("*"*30)

# Gerçek content ve wording değerleri
content_true = y_test['content'].values
wording_true = y_test['wording'].values

# Tahmin edilen content ve wording değerleri (örnek olarak rastgele tahminler kullanıldı)
content_pred = np.random.rand(len(y_test))
wording_pred = np.random.rand(len(y_test))

# `compt_score` fonksiyonunu kullanarak ortalama RMSE skorunu hesaplayın
average_rmse = compt_score(content_true, content_pred, wording_true, wording_pred)

# Sonucu yazdırın
print("Ortalama RMSE:", average_rmse)

MSE: 3.4131609761588807e+21
R-squared: -3.30938923322951e+21
Content RMSE: 1.1926486609375508
Wording RMSE: 1.1985810331041258
MCRMSE: 1.1956148470208383
******************************
Ortalama RMSE: 1.2027654277826363


# Ufak deneme

In [10]:
# Content ve wording tahminlerinin ortalamasını alın
average_content = np.mean(content_pred)
average_wording = np.mean(wording_pred)

# Mevcut "sub" dosyasına tahminleri ekleyin
sub['content'] = average_content
sub['wording'] = average_wording

# Güncellenmiş gönderim dosyasını kaydedin
sub.to_csv('submission.csv', index=False)

In [11]:
sub

,student_id,content,wording
0,000000ffffff,0.503214,0.481397
1,111111eeeeee,0.503214,0.481397
2,222222cccccc,0.503214,0.481397
3,333333dddddd,0.503214,0.481397
